In [1]:
import copy
import random as rd
import csv
import math

In [15]:
walking_speed = 1  # m/s

routeTable = {}
roadLengthTable = {}
arrivalTime = {}
arrivalProfile = {}

#Route table initialization
with open('roadCalcs_routeTable_post.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        profileId = row[0]
        popSplit = float(row[2])
        route = list(filter(lambda x:x!='',row[3:]))
        if profileId not in routeTable:
            routeTable[profileId] = []
        routeTable[profileId].append(
            {
                'popSplit': popSplit,
                'route': route
            }
        )

#Route length initialization
with open('roadCalcs_roadLengthTable.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        roadProfile = row[0]
        length = float(row[1])
        roadLengthTable[roadProfile] = {
            'length': length,
            'walking_time': round(length / walking_speed) # Calculate walking time in seconds
        }

#population and arrival initialization
with open('roadCalcs_arrivalTime.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        profileId = row[0]
        time = float(row[3])
        population = int(row[2])
        arrivalTime[profileId] = {
            'population': population,
            'time': time
        }

#arrival profile
with open('roadCalcs_arrivalProfile.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    data = list(reader)
    headers = data[0]
    arrivalData = data[1:]

    def pctFormatter(pct):
        try:
            return abs(float(pct))
        except:
            return abs(float(str(pct.split('%')[0])))/100

    for row in arrivalData:
        profileId = row[0]
        timeProfile = row[2:]
        arrivalProfile[profileId] = {}
        for idx,pct in enumerate(timeProfile):
            arrivalProfile[profileId][(idx*3600)] = 0 if pct == '' else pctFormatter(pct)


In [11]:
def round_down(num, divisor):
    return num - (num%divisor)

In [13]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        timeToSpendAtCurNode = self.node_detail[self.curNode]['walking_time']
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
    
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [16]:
print(routeTable) #node paths
print(roadLengthTable)
print(arrivalTime)
print(arrivalProfile)

{'1': [{'popSplit': 0.75, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['C']}, {'popSplit': 0.2, 'route': ['C', 'D', 'A4', 'A3', 'A2', 'A1']}], '2': [{'popSplit': 0.75, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['C']}, {'popSplit': 0.2, 'route': ['C', 'D', 'A4', 'A3', 'A2', 'A1']}], '3': [{'popSplit': 0.75, 'route': ['F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.2, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '4': [{'popSplit': 0.75, 'route': ['F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.2, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '5': [{'popSplit': 0.75, 'route': ['F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.2, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '6': [{'popSplit': 0.75, 'route': ['F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.2, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '7': [{'popSplit': 1.0, 'route': []}], '8': [{'popSplit': 1.0, 'route': []}

In [40]:
startingTime = 0*3600 #7pm in seconds
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time
simulationTime = 86400+startingTime #seconds
#Escalators
peoplePerHour = 1500


#keep track of time when people arrive at a node, where they are and when they should leave.

arrivalRatePerSecond = peoplePerHour / 3600
numberOfPeopleGenerated = {}
numberOfPeople = {}
peopleList = []
peopleEnd = 0
nodeOccupancies = {}


for profile in routeTable:
    numberOfPeopleGenerated[profile] = {}
    for routeId,routes in enumerate(routeTable[profile]):
        numberOfPeopleGenerated[profile][routeId] = 0


print(numberOfPeopleGenerated)
for curTime in range(startingTime,simulationTime,1):
    timeBucket = round_down(curTime, 3600)
    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        nodeOccupancies[curTime]['end'] = 0
        for key in roadLengthTable:
            nodeOccupancies[curTime][key] = 0

#loop through profiles and get the number of people
    for profile in arrivalProfile:
        if profile not in routeTable:
            continue
        for routeId, routes in enumerate(routeTable[profile]):

            numberOfArrivals = int(arrivalProfile[profile][timeBucket]*routes['popSplit']*arrivalTime[profile]['population'])
            # print(numberOfArrivals,profile,routeId,timeBucket)
            if curTime < timeBucket+arrivalTime[profile]['time']:
                rateOfPeople = (numberOfArrivals/arrivalTime[profile]['time'])
                numberOfPeopleGenerated[profile][routeId] += rateOfPeople

                profileNodePaths = routes['route']
                if len(profileNodePaths) == 0:
                    continue
                nodeDetails = roadLengthTable
                personArrivalAmount = math.floor(numberOfPeopleGenerated[profile][routeId])
                
        ############################################################################################################
                ## Lift Arrival Methodd
                # offset = idx * 12
                # if (curTime+offset)%nodeRoundTime[paths] == 0:# People Arrive at node 1
                #     timeBucket = round_down(curTime, 3600)
                #     liftOccupancy = int(pathLiftCapacity[paths] * liftCapacityArrivalRate[timeBucket])
                #     for personIndex in range(liftOccupancy):
                #         peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))
        ##############################################################################################################        

                for personIndex in range(int(personArrivalAmount)):
                    peopleList.append(person(len(peopleList),curTime,profileNodePaths,nodeDetails))
            numberOfPeopleGenerated[profile][routeId]-=math.floor(numberOfPeopleGenerated[profile][routeId])

    newPeopleList = []
    for agentId, agent in enumerate(peopleList):
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode

        if agentLocation == 'end':
            peopleEnd += 1
            continue

        nodeOccupancies[curTime][agentLocation] += 1
        newPeopleList.append(agent)
    nodeOccupancies[curTime]['end'] = peopleEnd
    peopleList = newPeopleList

# print(nodeOccupancies)

{'1': {0: 0, 1: 0, 2: 0}, '2': {0: 0, 1: 0, 2: 0}, '3': {0: 0, 1: 0, 2: 0}, '4': {0: 0, 1: 0, 2: 0}, '5': {0: 0, 1: 0, 2: 0}, '6': {0: 0, 1: 0, 2: 0}, '7': {0: 0}, '8': {0: 0}, '9': {0: 0, 1: 0, 2: 0}, '10': {0: 0, 1: 0, 2: 0}, '11': {0: 0, 1: 0, 2: 0}, '12': {0: 0, 1: 0, 2: 0}, '13': {0: 0, 1: 0, 2: 0}, '14': {0: 0, 1: 0, 2: 0}, '15': {0: 0, 1: 0, 2: 0}, '16': {0: 0, 1: 0, 2: 0}, '17': {0: 0, 1: 0, 2: 0}, '18': {0: 0, 1: 0, 2: 0}, '19': {0: 0, 1: 0, 2: 0}, '20': {0: 0, 1: 0, 2: 0}, '21': {0: 0, 1: 0, 2: 0}, '22': {0: 0, 1: 0, 2: 0}, '23': {0: 0, 1: 0, 2: 0}, '24': {0: 0, 1: 0, 2: 0}, '25': {0: 0, 1: 0, 2: 0}, '26': {0: 0, 1: 0, 2: 0}, '27': {0: 0, 1: 0, 2: 0}, '28': {0: 0, 1: 0, 2: 0}, '29': {0: 0, 1: 0, 2: 0}, '30': {0: 0, 1: 0, 2: 0}, '31': {0: 0}, '32': {0: 0}, '33': {0: 0, 1: 0, 2: 0}, '34': {0: 0, 1: 0, 2: 0}}


In [41]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('roadCalcs_Output.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)